# RAG Self-Consistency
LLM의 확률적 특성을 이용해서, 여러번 답변을 생성하고, 그중에 가장 일관된 답변(다수결)을 채택해서 최종응답으로 사용하는 기법이다.

In [4]:
%pip install langchain langchain-openai sentence-transformers scikit-learn -Uq

Note: you may need to restart the kernel to use updated packages.


In [5]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY'] = os.getenv('langsmith_key')
os.environ['LANGSMITH_PROJECT'] = 'skn23-langchain'
os.environ['OPENAI_API_KEY'] = os.getenv("openai_key")

## 가상 벡터db 조회

In [6]:
# 간단한 벡터 DB 입출력 예제
from langchain_core.documents import Document

def retrieve_vectordb(query=None):
    return [
        Document(page_content="파리의 상징은 에펠탑이며, 1889년에 세워졌습니다."),  # 에펠탑 기본 정보
        Document(page_content="파리는 세느강을 따라 발달한 도시로, 루브르 박물관은 파리의 상징입니다."),  # 도시 구조 및 대표 박물관
        Document(page_content="파리는 연간 약 2천만 명의 관광객이 방문하는 세계적 관광 도시입니다. 많은 관광객이 파리의 상징인 개선문을 방문하고 있습니다.")  # 관광 규모 및 랜드마크
    ]

retrieve_vectordb()

[Document(metadata={}, page_content='파리의 상징은 에펠탑이며, 1889년에 세워졌습니다.'),
 Document(metadata={}, page_content='파리는 세느강을 따라 발달한 도시로, 루브르 박물관은 파리의 상징입니다.'),
 Document(metadata={}, page_content='파리는 연간 약 2천만 명의 관광객이 방문하는 세계적 관광 도시입니다. 많은 관광객이 파리의 상징인 개선문을 방문하고 있습니다.')]

## RAG Chain

In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import init_chat_model

prompt = PromptTemplate.from_template('''  # 여행 일정 생성 프롬프트
아래 주어진 문서를 참고해서 사용자의 [질문]에 대한 여행일정을 작성해주세요.

[검색된 문서]
{context}

[질문]
{question}

[지시사항]
- 답변은 **최종추천일정:**으로 시작하세요.
- 일자별 일정은 한문장으로 요약하세요.
- 불필요한 서술은 생략하고, 핵심일정만 나열하세요.
''')

llm = init_chat_model('gpt-4.1-mini', temperature=1, n=5)  # 답변 생성용 LLM 설정 (창의성 1, 한번에 5개 응답 생성)
output_parser = StrOutputParser()      # LLM 출력 문자열로 파싱

question = '파리의 역사, 관광지, 방문시기를 종합해서 3일 여행 일정을 추천해주세요.'

retrieved_docs = retrieve_vectordb(question)  # 더미 벡터 검색 수행
context = '\n\n'.join([doc.page_content for doc in retrieved_docs])  # retrieved_docs에서 page content만 빼서 하나의 텍스트로 병합

messages = prompt.format_prompt(context=context, question=question).to_messages()  # 프롬프트를 메시지로 변환
response = llm.generate([messages])  # 리스트로 감싸 5개 응답 생성
print(response)

generations=[[ChatGeneration(text='**최종추천일정:**  \n1일차: 에펠탑 방문 및 세느강 유람선 탑승으로 파리의 대표 경관 감상.  \n2일차: 루브르 박물관 관람과 개선문 및 샹젤리제 거리 산책.  \n3일차: 몽마르트 언덕과 사크레쾨르 대성당 방문 후 파리 역사 탐방.  \n\n방문시기는 봄~가을(4월~10월)이 관광과 날씨 모두 적합합니다.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='**최종추천일정:**  \n1일차: 에펠탑 방문 및 세느강 유람선 탑승으로 파리의 대표 경관 감상.  \n2일차: 루브르 박물관 관람과 개선문 및 샹젤리제 거리 산책.  \n3일차: 몽마르트 언덕과 사크레쾨르 대성당 방문 후 파리 역사 탐방.  \n\n방문시기는 봄~가을(4월~10월)이 관광과 날씨 모두 적합합니다.', additional_kwargs={'refusal': None}, response_metadata={'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c40d5-1387-7373-9143-3ff0b38ed4b5-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 215, 'output_tokens': 632, 'total_tokens': 847, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})), ChatGeneration(text='**최종추천일정:**\n\n1일차: 에펠탑 방문 후 세느강 유람선 탑승 및 주변 카페에서 휴식  \n2일차: 루브르 박물관 관람과 개선문, 샹젤리제 거리 산책  \n3일차: 몽마르트 언덕과 사크레쾨르 성당

In [11]:
candidates = [gen.text for gen in response.generations[0]]  # 생성된 5개 응답 텍스트만 추출

# 후보 일정 순회 출력
for cand in candidates:
    print(cand)
    print("=========================")
    print()

**최종추천일정:**  
1일차: 에펠탑 방문 및 세느강 유람선 탑승으로 파리의 대표 경관 감상.  
2일차: 루브르 박물관 관람과 개선문 및 샹젤리제 거리 산책.  
3일차: 몽마르트 언덕과 사크레쾨르 대성당 방문 후 파리 역사 탐방.  

방문시기는 봄~가을(4월~10월)이 관광과 날씨 모두 적합합니다.

**최종추천일정:**

1일차: 에펠탑 방문 후 세느강 유람선 탑승 및 주변 카페에서 휴식  
2일차: 루브르 박물관 관람과 개선문, 샹젤리제 거리 산책  
3일차: 몽마르트 언덕과 사크레쾨르 성당 방문하며 파리의 역사 탐방  

방문 시기는 봄 또는 가을로 쾌적한 날씨와 관광객 붐비는 시기를 피하는 것을 추천합니다.

**최종추천일정:**
1일차: 에펠탑 방문과 세느강 유람선 탑승으로 파리의 상징을 체험하세요.  
2일차: 루브르 박물관에서 예술과 역사를 감상하고 개선문 주변을 산책하세요.  
3일차: 파리 구시가지 및 몽마르트 언덕을 탐방하며 도시의 역사를 느껴보세요.  

방문시기는 봄(4~6월)이나 가을(9~10월)이 관광객이 적당하고 쾌적합니다.

**최종추천일정:**  
1일차: 에펠탑 방문 및 세느강 유람선 투어로 파리의 상징과 도시 전경 감상.  
2일차: 루브르 박물관 관람 후 개선문과 샹젤리제 거리 산책.  
3일차: 몽마르트 언덕 방문 및 지역 역사 탐방, 야간에는 세느강변 야경 감상.  

방문 시기는 날씨가 온화한 봄(4~6월)이나 가을(9~10월)을 추천합니다.

**최종추천일정:**

1일차: 에펠탑 방문 후 세느강 유람선 탑승, 개선문과 샹젤리제 거리 산책  
2일차: 루브르 박물관 관람과 인근 튀일리 정원 산책  
3일차: 몽마르트 언덕과 사크레쾨르 성당 방문, 파리 거리 카페 체험  

방문시기는 날씨가 좋은 봄(4~6월)이나 가을(9~10월)을 추천합니다.



### n개의 답변을 하나로 추출하기

In [ ]:
from langchain_core.output_parsers import BaseOutputParser  # 출력 파서 베이스 클래스
from sentence_transformers import SentenceTransformer  # 문장 임베딩 모델
from pydantic import Field          # 필드 메타/유효성 정의
from sklearn.cluster import KMeans  # KMeans 클러스터링(군집화)
from collections import Counter     # 라벨 다수결 계산
import numpy as np                  # 수치 연산

class RobustSelfConsistencyParser(BaseOutputParser):
    n_clusters: int = Field(default=2)  # 클러스터 개수(유효성 검사 포함)
    encoder: object = Field(default=SentenceTransformer('all-MiniLM-L6-v2'))  # 문장 임베딩 인코더

    def parse(self, generations: list[str]) -> str:
        # 1. 임베딩
        embeddings = self.encoder.encode(generations)  # 후보 문장들을 벡터로 변환
        print(embeddings.shape)  # (후보수, 임베딩차원)

        # 2. 클러스터링(KMeans)
        kmeans = KMeans(n_clusters=self.n_clusters, random_state=42)
        kmeans.fit(embeddings)  # 임베딩 기반 군집화
        print(kmeans.labels_)   # 각 후보의 클러스터 라벨

        # 3. 다수결 투표
        counts = Counter(kmeans.labels_)  # 라벨별 개수
        target_label = max(counts, key=counts.get)  # 가장 많은 라벨 선택
        target_indices = np.where(kmeans.labels_ == target_label)[0]  # 다수 라벨에 속한 후보 인덱스 추출
        print(target_label)
        print(target_indices)

        # 4. 대표 답변 선택 (중심점에 가장 가까운 후보)
        target_cnetroid = kmeans.cluster_centers_[target_label]  # 다수 클러스터의 중심점
        distances = np.linalg.norm(embeddings[target_indices] - target_cnetroid, axis=1)  # 각 후보에서 중심점까지의 거리
        representive_idx = np.argmin(distances)  # 가장 가까운 후보 인덱스
        return generations[target_indices[representive_idx]]  # 대표 답변 반환

parser = RobustSelfConsistencyParser()   # 파서 생성
final_answer = parser.parse(candidates)  # 대표답변 생성
print(f'최종 답변 : {final_answer}')

(5, 384)
[0 0 0 1 1]
0
[0 1 2]
최종 답변 : **최종추천일정:**

1일차: 에펠탑 방문 후 세느강 유람선 탑승 및 주변 카페에서 휴식  
2일차: 루브르 박물관 관람과 개선문, 샹젤리제 거리 산책  
3일차: 몽마르트 언덕과 사크레쾨르 성당 방문하며 파리의 역사 탐방  

방문 시기는 봄 또는 가을로 쾌적한 날씨와 관광객 붐비는 시기를 피하는 것을 추천합니다.


In [ ]:
# 여행 일정 후보를 여러 개 생성 후, 클러스터링 기반 Self-Consistency로 최종 답변을 선택하는 함수
def travel_planner(question, verbose=False):
    retrieved_docs = retrieve_vectordb(question)  # 질문과 관련된 문서 검색
    context = '\n\n'.join([doc.page_content for doc in retrieved_docs])  # 검색된 문서를 하나의 컨텍스트로 병합

    messages = prompt.format_prompt(context=context, question=question).to_messages()  # 프롬프트를 메시지로 변환
    
    response = llm.generate([messages])  # 리스트로 감싸 5개 응답 생성
    candidates = [gen.text for gen in response.generations[0]]  # 후보들의 텍스트만 추출

    if verbose:  # 후보 일정들을 출력하곳싶을 때
        for idx, cand in enumerate(candidates):
            print(f'{idx + 1}')
            print(cand)  # 후보 일정
            print("=========================")
            print()

    parser = RobustSelfConsistencyParser()  # 대표답변 고르는 파서
    return parser.parse(candidates)         # 최종 일정(대표 답변) 반환

question = '파리의 역사, 관광지, 방문시기를 종합해서 3일 여행 일정을 추천해주세요.'
response = travel_planner(question, verbose=True)
print(f'최종답변 : {response}')

1
**최종추천일정:**

1일차: 에펠탑 방문 및 세느강 유람선 투어로 파리의 상징과 경관 감상  
2일차: 루브르 박물관 관람과 개선문, 샹젤리제 거리 산책  
3일차: 역사적인 마레 지구 탐방과 노트르담 성당 방문  

방문 시기는 봄(4~6월) 또는 가을(9~10월)을 권장합니다.

2
**최종추천일정:**  
1일차: 에펠탑 방문 및 세느강 유람선 탑승으로 파리 상징 체험,  
2일차: 루브르 박물관 관람 후 개선문과 샹젤리제 거리 산책,  
3일차: 파리 역사 탐방을 위한 노트르담 대성당 방문 및 인근 마레지구 카페 거리 산책.  

방문시기는 관광객이 혼잡하지 않은 봄(4~6월)이나 가을(9~10월)을 권장합니다.

3
**최종추천일정:**  
1일차: 에펠탑 방문 및 세느강 유람선 탑승, 주변 카페에서 휴식  
2일차: 루브르 박물관 관람 후 튈르리 정원 산책, 개선문 및 샹젤리제 거리 걸으며 야경 감상  
3일차: 몽마르트 언덕과 사크레쾨르 대성당 방문, 파리 역사박물관 또는 주변 마을 탐방  

방문시기는 봄(4~6월) 또는 가을(9~10월)을 추천하며, 쾌적한 날씨와 관광객 적은 시기입니다.

4
**최종추천일정:**  
1일차: 에펠탑과 세느강 주변 산책 및 야경 감상,  
2일차: 루브르 박물관 관람과 개선문 방문, 샹젤리제 거리 산책,  
3일차: 파리 역사 탐방 및 주변 카페에서 여유로운 휴식, 봄(4~6월) 또는 가을(9~11월) 방문 추천.

5
**최종추천일정:**

1일차: 에펠탑 방문과 세느강 유람선 투어로 파리 역사와 경치를 즐기기  
2일차: 루브르 박물관에서 세계적 예술작품 감상 후 개선문과 샹젤리제 거리 산책  
3일차: 몽마르트 언덕과 사크레쾨르 성당 방문, 파리의 전통적인 분위기 체험  

방문 시기는 날씨가 온화한 봄(4~6월)이나 가을(9~10월)을 추천합니다.

(5, 384)
[0 0 0 1 0]
0
[0 1 2 4]
최종답변 : **최종추천일정:**

1일차: 에펠탑 방문과 세느강 유람선 투어로 파리 역사